## Installation and deployment of 5G RAN (gNB) of Open Air Interface (OAI)
## Objectives
- Install and configure the the OAI gNB 
- Deploy a gNB together a previous configured 5GC
- Inspec gNB status in 5GC Core

At the end of a set of three hands-on (this is the second one), the goal is to run the simulated OAI scenario, which consists of the 5G core (5GC), gNB in CU/DU split mode, and user equipment (UE), according to the image below: 

![5GC_01](./FIGS/H01_5GCore_UNI_III/5GC_deploy.jpg)

Now, we are going to install, configure, deploy and interact to gNB with CU/DU split mode. 


## gNB Installation

To install OAI RAN 5G, go back to /home/dco1020 and follow this tutorial.

**Step 0:** go back to /home/dco1020. In a Linux Terminal, type:

sudo su

cd /home/dco1020


**Step 1:** Install dependencies

sudo add-apt-repository ppa:ettusresearch/uhd

sudo apt-get install libuhd-dev uhd-host -y

sudo apt-get install libuhd4.2.0 -y

sudo apt-get install libuhd4.4.0 -y

**Step 2:** Fix problem in the libuhd. When you run the commands below, if you get the error "cannot access archive", you can continue to the next step.

sudo dpkg -i --force-overwrite /var/cache/apt/archives/libuhd4.2.0_4.2.0.1-0ubuntu1~focal1_amd64.deb

sudo dpkg -i --force-overwrite /var/cache/apt/archives/libuhd4.4.0_4.4.0.0-0ubuntu1~focal1_amd64.deb

**Step 3:** Clone OpenAirInterface 5G RAN repository

git clone https://gitlab.eurecom.fr/oai/openairinterface5g.git

cd /home/dco1020/openairinterface5g

git checkout v2.0.0

**Step 4:** Run the installation script

source oaienv

cd cmake_targets

sudo ./build_oai -I -w USRP

sudo ./build_oai -I --install-optional-packages -w USRP

sudo ./build_oai --gNB --eNB --UE --nrUE -w USRP

obs: you may find Wireshark configuration screen, select "yes" to continue.


## gNB Configuration

In a Linux Terminal, follow the steps below:

**Step 1:** Configure the PLMN and TAC values

The Public Land Mobile Network (PLMN) and Tracking Area Code (TAC) will be configured in the CU (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/cu_gnb.conf) and DU (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/du_gnb.conf) configuration files. 

Figure below shows the part of the code that must be modified in the CU and DU files (lines 17 and 18). The parameters shown in figure should be modified according to the values present in docker compose in the "oai-amf" session (/home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml). 

![gNB_config_01](./FIGS/H02_5GRAN_UNI_III/gNB_config_01.png)


Figure below shows the parameters that should be checked and passed to the CU and DU configuration files: PLMN_SUPPORT_TAC (referring to "tracking_area_code"), PLMN_SUPPORT_MCC (referring to "mcc"), PLMN_SUPPORT_MNC (referring to "mnc"), SST_0 (referring to "sst") and the SD_0 (referring to "sd"). If the "sd" parameter is not in the CU and DU configuration files, simply add it according to line 18 in previous figure.

![gNB_config_02](./FIGS/H02_5GRAN_UNI_III/gNB_config_02.png)

**Step 2:**  Configure the network interfaces

This part you will configure the N2 interface in (/home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/cu_gnb.conf), such as in figure below. This interface is responsible to connect the CU and the AMF.

![gNB_config_03](./FIGS/H02_5GRAN_UNI_III/gNB_config_03.png)

You need to verify/change the IP addresses. You can find the OAI-AMF IP address inside /home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml, such as in figure below. You must get this IP address and put it in the "amf_ip_address" parameter, in the "ipv4" field (such as in previous figure).

![gNB_config_04](./FIGS/H02_5GRAN_UNI_III/gNB_config_04.png)

**Step 1:** to check a virtual interface (demo-oai), type:

ifconfig -a

You should find a virtual interface (demo-oai) (as shown in figure below). This interface serves to connect the CU with OAI-AMF.  You must get its IP address and name to configure the N2 interface such as two figures before. 

![gNB_config_05](./FIGS/H02_5GRAN_UNI_III/gNB_config_05.png)

## Deployment

For this task, open another terminal using crtl+Alt+T.

**Step 1:** Core deployment




![gNB_config_06](./FIGS/H02_5GRAN_UNI_III/gNB_config_06.png)

## Verificação dos Pods com linha de comando e com o Portainer

The 5GC components were installed as docker containers. A container is a standard unit of software that packages up code and all its dependencies so the application runs quickly and reliably from one computing environment to another. A Docker container image is a lightweight, standalone, executable package of software that includes everything needed to run an application: code, runtime, system tools, system libraries and settings. Container images become containers at runtime and in the case of Docker containers – images become containers when they run on Docker Engine. Available for both Linux and Windows-based applications, containerized software will always run the same, regardless of the infrastructure. Containers isolate software from its environment and ensure that it works uniformly despite differences for instance between development and staging.

Containers and virtual machines have similar resource isolation and allocation benefits, but function differently because containers virtualize the operating system instead of hardware. Containers are more portable and efficient. Figure bellow illustrates the difference between VMs and containers.

![5GC_deploy_04](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_04.png)

**Step 1**: to see all 5GC docker containers by command line you could type the following command in a terminal:

docker ps -a

The results are as bellow:

![5GC_deploy_03](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_03.png)

You can see all virtual functions that comprise the 5GC.


**Step 2**: let's logs of a running docker container, for instance, let's see logs of the 5GC's AMF container. In a terminal , type:

docker logs -f 2de7bfa9afa4

You will see a screen similar to the figure bellow.

![5GC_deploy_05](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_05.png)


You can find the same results with the command:

docker logs -f oai-amf

**Step 3**: you can also see the hardware resources all containers are consuming. In a terminal , type:

docker container stats

You will see a screen similar to the figure bellow.

![5GC_deploy_06](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_06.png)

To return to terminal, type: crtl+c

**Step 4**: let's execute a command inside a running docker container, for instance, let's open a bash inside the 5GC's AMF container and see the processes in execution. See in figures above that **2de7bfa9afa4** is the container ID of the 5GC's AMF container. So, in a terminal , type (ps.: realize that you need to see the containerID of 5GC's AMF installed in your machine):

docker exec -it 2de7bfa9afa4 bash

You will see a screen similiar to the figure bellow.

![5GC_deploy_07](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_07.png)

As you have a bash shell inside the container, you can type bash commands. Let's see all processes running by this container. Type:

top

You will see a screen similiar to the figure bellow.

![5GC_deploy_08](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_08.png)

To return to terminal, type: crtl+c

**Step 5 (optional)**: To stop the core network you can use:

python3 core-network.py --type stop-basic --scenario 1


A good Docker Cheat Sheet can be found [here](https://docs.docker.com/get-started/docker_cheatsheet.pdf).


**Step 5 (optional)**: there is a plenty of graphical tool to manager, see and operate dockers. Let's install and use Portainer. To install it, first, create the volume that Portainer Server will use to store its database. Ia a terminal, type:

docker volume create portainer_data

**Step 6 (optional)**: Then, download and install the Portainer Server container:

docker run -d -p 8000:8000 -p 9443:9443 --name portainer --restart=always -v /var/run/docker.sock:/var/run/docker.sock -v portainer_data:/data portainer/portainer-ce:latest

**Step 7 (optional)**: Now that the installation is complete, you can log into your Portainer Server instance by opening a web browser and going to:

https://localhost:9443


**Step 8 (optional)**: You need to create a user. For this, you will see a screen similar to:

![5GC_port_8](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_08.png)

**Step 9 (optional)**: Now, you can see the initial screen and you could proceedd using the local environment. Please, click on **Get Started**.

![5GC_port_9](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_09.png)

**Step 10 (optional)**: then, you can see the screen with the local environment. Click on it.

![5GC_port_10](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_10.png)

**Step 11 (optional)**: Now, you could see the environment dashboard with all information about your Portainer. Let's see all Containers clicking on Containers menu as indicating below.

![5GC_port_11](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_11.png)

**Step 12 (optional)**: you can see all containers, their status and a menu of quick actions. 

![5GC_port_12](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_12.png)

**Step 13 (optional)**: To see the AMF container logs, please click on icone indicated in the figure below.

![5GC_port_13](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_13.png)

**Step 14 (optional)**: Finally, you can see AMF logs (note that you can filter and download it).

![5GC_port_14](./FIGS/H01_5GCore_UNI_III/5GC_deploy_port_14.png)

An interesting video about Portainer tool is: [Controle seus containers de maneira SIMPLES com o Portainer - Diolinux Labs](https://www.youtube.com/watch?v=q0LrLN4fylw),

# Deliverable 01:
See the log of all docker containers of 5GC and identify the following information:

- GUAMI (MCC, MNC, Region ID, AMF Set ID, AMF pointer):
- Slice Support: what are SST, SD supported? Explain those parameters. Indicate the 5QI of whoich slice.
- What is the N2 IP Address and port?
- What is the NRF IP Address and port?
- What is the AUSF IP Address and port?
- What is the AMF IP Address and port?
- What is the N4 IP Address and port?

Show the logs that you see all information.